<h1><center>Laboratorio 4: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Joaquín Roa
- Nombre de alumno 2: Matías Sepúlveda


### **Link de repositorio de GitHub:** `https://github.com/jroallanos/LDPCPCDD19`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Clasificación con texto.
- Clasificación en `scikit-learn`.
- Modelos a través del uso de `pipeline`.
- Optimización de modelos usando `GridSearchCV`.

## Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Crear modelos de clasificación de texto.
- Optimizar la clasificación de texto usando wordclouds.
- Usar herramientas de visualización de texto como las wordclouds.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jroallanos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jroallanos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jroallanos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://static.wikia.nocookie.net/p__/images/a/a2/Bat-Cow.jpg/revision/latest?cb=20180108185037&path-prefix=protagonist" width="350">
</p>

En vez de estar oprotunamente desarrollando las tareas y las evaluaciones correspondientes al curso, su profesor de catedra y su auxiliar discuten acerca la alineación del personaje de ficción *Bat-Cow*. 

El cuerpo docente, no logra ponerse de acuerdo acerca de la alineación del personaje, es decir, si lucha junto a las fuerzas del bien, si neutral a cualquier eventualidad o derechamente es un villano.
El auxiliar plantea (de forma superficial) que *Bat-cow* posee una siniestra mirada, común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor (*se cree filósofo... y*) plantea que tal como los humanos no nacen malos, no existe motivo por el cual un rumiante humanizado con superpoderes deba serlo.

Sin embargo, ambos concuerdan en es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para iniciar este laboratorio, cargue los dataset señalados y visualice a través de la función `head` los atributos que poseen cada uno de los dataset.


In [31]:
#dir = '/content/'
#df_comics = pd.read_csv(dir+'df_comics.csv')
#df_comics_no_label = pd.read_csv(dir+'comics_no_label.csv')
df_comics = pd.read_csv('/Users/jroallanos/Desktop/Laboratorio de Programación Científica para Ciencia de Datos/labs/lab4/df_comics.csv')
df_comics_no_label = pd.read_csv("/Users/jroallanos/Desktop/Laboratorio de Programación Científica para Ciencia de Datos/labs/lab4/comics_no_label.csv")
df_comics = df_comics.drop(columns=['Unnamed: 0'])
df_comics_no_label = df_comics_no_label.drop(columns=['Unnamed: 0'])

## 1.1 Obtención de Features [2 puntos]

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Su primera tarea consiste en generar un vector de características para el atributo `history_text`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset (si un personaje tiene este atributo nulo, elimínelo). Luego, para obtener características de texto aplique el modelo de conteo `bag of words` de la siguiente forma:

- Utilice `CountVectorized` junto al tokenizador (que le proveemos) `LemmaTokenizer`.
- Obtenga caracteristicas de los 1-gramas y 2-gramas del texto (ver clase).
- Fijar un maximo de 10.000 caracteristicas para el vector de salida.

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

In [32]:
df_comics = df_comics.dropna(axis = 0, subset=['history_text'])
df_comics_no_label = df_comics_no_label.dropna(axis = 0, subset=['history_text'])

In [33]:
df_comics

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1363,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,100,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1364,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1365,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
df_comics_no_label

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,75,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
6,A.M.A.Z.O. (CW),NaN,NaN,22,The Anti Meta-human Adaptive Zootomic Organism...,"With its metallic body, it easily overpowered ...",90,100,100,100,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,90,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
80,Wendigo,Wendigo,NaN,6,The Wendigo are the result of an ancient curse...,NaN,65,80,75,75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
81,XS (CW),Nora West-Allen,NaN,19,"According to Eobard Thawne, the daughter of Ba...",Nora possesses a natural connection to the Spe...,90,55,100,60,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
82,Yondu (MCU),Yondu,NaN,7,"Yondu Udonta was a Centaurian, leader of a key...","Yondu Udonta was a member of the Centaurians, ...",85,25,40,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
stop_words = stopwords.words('english')

class LemmaTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

In [5]:
preprocessing_transformer = ColumnTransformer(
    transformers = [
        ('vectorizer', CountVectorizer(tokenizer = LemmaTokenizer(), max_features = 10000 , ngram_range = (1,2)),  'history_text'),
        ('MinMaxScaler', MinMaxScaler(), ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])])

## 1.2 Diseño de Pipeline y  Primer Entrenamiento [1.5 puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


A continuación, genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, añadiendo un reductor de dimensionalidad llamado `TruncatedSVD()` ajustando el número de componentes en 1000 (este reducto de dimensionalidad es similar al PCA pero funciona para vectores dispersos) y un clasificador `MultinomialNB()` por defecto.  Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde el etiquetado vendrá dado por el atributo `alignment`. Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**Nota:** Debido al desbalance que existe entre las clases, puede ser util aplicar método de [`imbalanced-learn`](https://github.com/scikit-learn-contrib/imbalanced-learn) como RandomOverSampler sobre los datos de entrenamiento. 

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1,aplicar un reductor de dimensionalidad `TruncatedSVD` y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline.
- [ ] (Opcional - **0.5 bonus**) Utilizar técnicas de Sampling para balancear los datos de entrenamiento.


In [6]:
pipeline_bat_cow = Pipeline(
    steps = [
        ("Preprocessing", preprocessing_transformer),
        ("clf", MultinomialNB()),
    ]
)

In [7]:
X_train, X_test,y_train,y_test=train_test_split(
    df_comics.loc[:,['history_text', 'intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']],
    df_comics.alignment,
    shuffle=True,
    test_size=0.3,
    random_state=19
)

In [8]:
int_columns = ['history_text','intelligence_score',	'strength_score',	'speed_score',	
               'durability_score',	'power_score',	'combat_score']

X_train = X_train[int_columns] 
over = RandomOverSampler(random_state=42)
X_train, y_train = over.fit_resample(X_train, y_train)
X_train = pd.DataFrame(X_train, columns=int_columns)

In [9]:
pipeline_bat_cow.fit(X_train, y_train)

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('vectorizer',
                                                  CountVectorizer(max_features=10000,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.LemmaTokenizer object at 0x7ff89b7c9eb0>),
                                                  'history_text'),
                                                 ('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                            

In [10]:
y_pred = pipeline_bat_cow.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.56      0.50      0.53       132
        Good       0.68      0.78      0.73       216
     Neutral       0.21      0.11      0.14        38

    accuracy                           0.62       386
   macro avg       0.48      0.46      0.47       386
weighted avg       0.59      0.62      0.60       386



## 1.3 Entrenamiento con Grid Search [2 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen una búsqueda de grilla de los mejores hiperparámetros utilizando `GridSearchCV`. 

Para esto, se le solicita que defina al menos 3 configuraciones de hiperparámetros e intente obtener mejores resultados que los obtenidos en la sección anterior. 

A continuación, un ejemplo de parametros para GridSearch:

```python
params = [
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator1.
  # en este caso se entrenará el clasificador 1 con combinaciones de todos los 
  # parámetros de bow__max_features, bow__ngram_range, clf__n_estimators 
  # y se seleccionará la mejor combinación.
  {
  'bow__max_features': [5000, 10000, ...],
  'bow__ngram_range': [(1, 1), (1, 2), (1,3)],
  ...,
  'clf': [classificator1()],
  'clf__n_estimators': [200]
  },
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator2:
  {'clf': [classificator2()],
   'clf__penalty': ['ovr'],
   'clf__multi_class': ['liblinear']
  },
  # esta es la configuración de una busqueda en particular
  # con el clasificador classificator3:
  {'clf': [classificator3()]
  }
             ]
```

Además, note que puede obtener todos los parámetros configurables de un pipeline invocando sobre este el método `.get_params()`.

**Nota:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda dejar corriendo el código y tomarse un tecito.

In [11]:
pipeline_bat_cow.get_params()

{'memory': None,
 'steps': [('Preprocessing',
   ColumnTransformer(transformers=[('vectorizer',
                                    CountVectorizer(max_features=10000,
                                                    ngram_range=(1, 2),
                                                    tokenizer=<__main__.LemmaTokenizer object at 0x7ff89b7c9eb0>),
                                    'history_text'),
                                   ('MinMaxScaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('clf', MultinomialNB())],
 'verbose': False,
 'Preprocessing': ColumnTransformer(transformers=[('vectorizer',
                                  CountVectorizer(max_features=10000,
                                                  ngram_range=(1, 2),
                                                 

In [16]:
params = [{
  'Preprocessing__vectorizer__max_features': [5000, 10000, 20000],
  'Preprocessing__vectorizer__ngram_range': [(1, 1), (1, 2), (1,3)],
  'clf': [RandomForestClassifier()],
  'clf__max_depth': [10, 20, 30],
  'clf__n_estimators': [600, 800, 1000]
  }, 
  {
  'clf': [MultinomialNB()],
  'clf__alpha': [0.1, 0.5],
  }, 
  {
  'clf': [DecisionTreeClassifier()],
  'clf__criterion': ['gini', 'entropy'],
  'clf__max_depth': [6, 8, 12]
  }]

In [17]:
gs = GridSearchCV(pipeline_bat_cow, params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('Preprocessing',
                                        ColumnTransformer(transformers=[('vectorizer',
                                                                         CountVectorizer(max_features=10000,
                                                                                         ngram_range=(1,
                                                                                                      2),
                                                                                         tokenizer=<__main__.LemmaTokenizer object at 0x7ff89b7c9eb0>),
                                                                         'history_text'),
                                                                        ('MinMaxScaler',
                                                                         MinMaxScaler(),
                                                                         ['intelligence_score',
                    

In [18]:
gs.best_params_

{'Preprocessing__vectorizer__max_features': 5000,
 'Preprocessing__vectorizer__ngram_range': (1, 1),
 'clf': RandomForestClassifier(max_depth=30, n_estimators=600),
 'clf__max_depth': 30,
 'clf__n_estimators': 600}

In [19]:
y_pred2 = gs.predict(X_test)
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

         Bad       0.67      0.37      0.48       132
        Good       0.64      0.91      0.75       216
     Neutral       0.33      0.03      0.05        38

    accuracy                           0.64       386
   macro avg       0.55      0.44      0.43       386
weighted avg       0.62      0.64      0.59       386



#### 1.3.1 Mejor configuración [0.5]

Comente cual fue la mejor configuración obtenida por Grid Search y por qué cree que esta fue la mejor.

La mejor configuración obtenida por GridSearch que pudimos encontrar es la entregada finalmente (hicimos varias iteraciones) compuesta por Random Forest, MultinomialNB y Árboles de Decisión. Esta configuración entrega una precisión 'macro' de $51\%$ y 'weighted' de $62\%$, un recall 'macro' de $49\%$ y 'weighted' de $65\%$ y un F-measure 'macro' de $50\%$ y 'weighted' de $63\%$. Estas son las mejores métricas que pudimos encontrar probando diferentes configuraciones de hiperparámetros y combinaciones de métodos predictivos, sujetos a la restricción de que demora muchísimo correr estos modelos en nuestros computadores. Los modelos presentan un bajo desempeño pero mejoran los obtenidos en la sección anterior de la tarea.

## 1.4 Predicción del datos sin etiquetado

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


Llego el momento de predecir cual es la verdadera alineación de `Batcow`. Para esto, deben escoger el mejor pipeline obtenido en las secciones anteriores y predecir la alineación de todos los datos presentes en `df_comics_no_label`.Luego, anexen las alineaciones obtenidas a su correspondiente columna  del dataframe original (atributo `alignment`) y busquen a los flamantes personajes `Batcow`, `Vergil`, y `Gorilla Girl'. Presente los resultados en un `Dataframe`.

**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

In [48]:
alignment = gs.predict(df_comics_no_label[int_columns])
df_comics_no_label.alignment=alignment

In [49]:
df_comics_no_label[(df_comics_no_label["name"] == "Batcow") | (df_comics_no_label["name"] == "Vergil")| (df_comics_no_label["name"] == "Gorilla Girl")]

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
16,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,60,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
78,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,90,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
83,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
df_comics_no_label[(df_comics_no_label["name"] == "Batcow") | (df_comics_no_label["name"] == "Vergil")| (df_comics_no_label["name"] == "Gorilla Girl")].alignment

16    Good
40    Good
78     Bad
83    Good
Name: alignment, dtype: object

El mejor pipeline considera que la Bativaca es una superheroína. Lo probamos con varios pipeline y siempre nos dio este resultado, por lo que concluimos que la Bativaca es una superheroína, o mejor dicho, una supervaca :)

### Wordclouds [Opcional- 0,5] 

Una buena pero informal forma de comunicar los resultados del trabajo con texto es generar Wordclouds. Este tipo de visualizaciones nos informan de forma gráfica cuales son las palabras más frecuentes según el tamaño de estas al ser posicionadas en un lienzo.
<center>
<img alt='Ejemplo de una Wordcloud de Starwars' src='https://amueller.github.io/word_cloud/_images/sphx_glr_a_new_hope_001.png' width=400/>

Ejemplo de una Wordcloud de Starwars

</center>
Dicho esto, como equipo docente nos encantaría conocer cuales son las palabras que caracterizan tanto a heroes como neutrales y a villanos y cuales son sus principales diferencias. Por esta razón, les pedimos como última tarea generar una wordcloud con las historias de cada personaje según cada alineamiento (clase). Pueden ocupar el dataset completo para esto. 


**Nota:** Recuerde eliminar las stopwords. Guías completas para generar las wordclouds, eliminar las stopwords y configurar los parámetros de las nubes creadas pueden ser encontradas en su [documentación oficial](https://amueller.github.io/word_cloud/) y en [datacamp](https://www.datacamp.com/community/tutorials/wordcloud-python).





In [ ]:
#### Wordcloud para heroes ####

In [ ]:
#### Wordcloud para neutrales ####

In [ ]:
#### Wordcloud para villanos ####

Comente las principales diferencias entre las tres wordclouds.
¿Hay palabras que caracterizen a los grupos y que no aparezcan en los otros?

---> Comente aquí <---

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>